<a href="https://colab.research.google.com/github/akan72/comp590/blob/master/flask/src/notebooks/SVM_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://developers.google.com/earth-engine/classification
import ee

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xAFsxAJlVwWt3bgkO-UL6SCWt-6vyzKqmTqfxsPf3LD1NcKBR5tK908

Successfully saved authorization token.


In [0]:
import folium
import matplotlib.pyplot as plt
from IPython.display import Image

# Folium Utilities 
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [0]:
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(
    landsat.filterDate('2018-01-01', '2018-12-31'),
    asFloat=True)

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443)
forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735)
nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486)
nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986)

polygons = ee.FeatureCollection([
  ee.Feature(nonForest1, {'class': 0}),
  ee.Feature(nonForest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
])

In [0]:
training = image.sampleRegions(
    collection=polygons,
    properties=['class'],
    scale=30)

classifier = ee.Classifier.svm(
    kernelType='RBF',
    gamma=.5,
    cost=10)

In [0]:
trained = classifier.train(training, 'class', bands)
classified = image.classify(trained)

In [21]:
my_map = folium.Map(
    location=[ -9.2399, -62.836],
    zoom_start=9
)

my_map.add_ee_layer(image, {'bands': ['B4', 'B3', 'B2'], 'max': .5, 'gamma': 2}, 'rags')
# my_map.add_ee_layer(polygons, {}, 'training data')
# my_map.add_ee_layer(classified, {min: 0, max: 1, palette: ['red', 'green']},
            #  'deforestation')
my_map.add_child(folium.LayerControl())